In [27]:
import ezodf
import networkx as nx
import matplotlib.pyplot as plt

ods_filename = "Sitzen-Felix.ods"

# Illustrate the relationship between students with 'student classes', 
# which are having name, prename, and class. But also a reference to the students they like
class Student:

    def __init__(self, prename: str, name: str, clazz: str):
        self.likes = None
        self.prename = prename
        self.name = name
        self.clazz = clazz

    def set_likes(self, students: []):
        self.likes = students

    def shortcut(self):
        return f"{self.prename},{self.name},{self.clazz}"
    
    

# Let's start with the creation of the student objects
# To save them we use a dictionary with string as key and the student object as value
def create_student_dict(filename: str):
    student_dict = dict()
    
    # Read document
    document = ezodf.opendoc(filename)
    sheet = document.sheets[0]

    # 1. Go through all rows and add all students of the first column to the dict
    for i, row in enumerate(sheet.rows()):
        # Skip header
        if i == 0:
            continue

        cell = row[0]
        # Skip empty fields
        if cell.value is None:
            continue

        # Remove the spaces and split it up in its pieces 
        without_spaces = cell.value.replace(" ", "")
        split: list[str] = without_spaces.split(",")

        # Save the new created student in the dict
        student = Student(split[0], split[1], split[2])
        student_dict[without_spaces] = student

    # 2.  Add the relationships between the others to the students
    for i, row in enumerate(sheet.rows()):
        # If a row is empty, skip 
        if i == 0 or row[0].value is None:
            continue

        # Get the current student of the row
        selected_student = student_dict.get(row[0].value.replace(" ", ""))

        # Search for his friends in the dict, and add them as reference to him
        liked_students = []
        for j, cell in enumerate(row[1:3]):
            # If there is a friend
            if cell.value is not None and cell != "x":
                # Look for him in the dict and add a reference to the student
                without_spaces = cell.value.replace(" ", "")
                liked_students.append(student_dict.get(without_spaces))
        
        selected_student.set_likes(liked_students)

    # Return the final dict
    return student_dict

# Create a dict with all relations between the students
student_dict = create_student_dict(ods_filename)

# Use networkx to create a graph layout
G = nx.Graph()

# Add all students form the dict as nodes
for key, value in student_dict.items():
    G.add_node(key)

# Create an edge for every student, which has selected another student
for key, value in student_dict.items():
    for i, student in enumerate(value.likes):
        # If there is a friend
        if student is not None:
            # If his friend already selected him as friend, mark it red, to know that there like each other
            # if only one student likes the other, mark the edge black
            if G.has_edge(key, student.shortcut()):
                G.add_edge(key, student.shortcut(), color='red')
            else:
                G.add_edge(key, student.shortcut(), color='black')

# Create a position attribute for networkx. K describes the distance between the nodes, where iterations means, the amount of iterations to take for the picture.
# https://stackoverflow.com/questions/14283341/how-to-increase-node-spacing-for-networkx-spring-layout 
pos = nx.spring_layout(G, k=0.1, iterations=50)

# Create the color attribute for networkx
colors = nx.get_edge_attributes(G,'color').values()

# print the number of nodes to check if the amount of students is correct
print(f"Number of nodes: {G.number_of_nodes()}")

# Plot the graph with a high resolution.
plt.figure(3, figsize=(96, 96))
# Draw the final graph
nx.draw(G, pos, edge_color=colors ,with_labels=True, font_size=20, node_size=150)
plt.show()




Number of nodes: 112
